In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# ---------------------------
# Output Directory Setup
# ---------------------------
output_dir = "/content/drive/MyDrive/Edge-Pop_0.5_2/"
os.makedirs(output_dir, exist_ok=True)
log_file = os.path.join(output_dir, "active_learning_log.csv")
weights_file = os.path.join(output_dir, "global_lenet5.weights.h5")
final_subnet_weights = os.path.join(output_dir, "final_subnet.weights.h5")
final_global_weights = os.path.join(output_dir, "final_global_lenet5_full_training.weights.h5")

# ---------------------------
# Edge-Popup Components
# ---------------------------

class GetSubnet(tf.keras.layers.Layer):
    def __init__(self, k):
        super(GetSubnet, self).__init__()
        self.k = k

    def call(self, scores):
        scores_flat = tf.reshape(scores, [-1])
        k_val = tf.cast(tf.size(scores_flat), tf.float32) * self.k
        k_val = tf.cast(k_val, tf.int32)
        k_val = tf.maximum(k_val, 1)

        topk_values, topk_indices = tf.math.top_k(scores_flat, k=k_val, sorted=False)
        mask_flat = tf.zeros_like(scores_flat)
        mask_flat = tf.tensor_scatter_nd_update(
            mask_flat, tf.expand_dims(topk_indices, 1), tf.ones_like(topk_values)
        )
        return tf.reshape(mask_flat, tf.shape(scores))

class SubnetConv2D(tf.keras.layers.Layer):
    def __init__(self, base_weights, base_bias, filters, kernel_size, strides=1, padding="same", k=0.5, use_bias=True):
        super(SubnetConv2D, self).__init__()
        self.k = k
        self.use_bias = use_bias
        self.strides = strides
        self.padding = padding.upper()

        self.base_weights = base_weights
        self.base_bias = base_bias
        self.get_subnet = GetSubnet(k)

        self.popup_scores = self.add_weight(
            name="popup_scores",
            shape=base_weights.shape,
            initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
            trainable=True,
        )

    def call(self, inputs):
        mask = self.get_subnet(tf.abs(self.popup_scores))
        masked_weights = self.base_weights * mask
        x = tf.nn.conv2d(inputs, masked_weights, strides=[1, self.strides, self.strides, 1], padding=self.padding)
        if self.use_bias and self.base_bias is not None:
            x = tf.nn.bias_add(x, self.base_bias)
        return x

def build_global_lenet5(input_shape=(32, 32, 3), num_classes=10):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(6, kernel_size=5, padding="same", activation='tanh')(inputs)
    x = layers.AveragePooling2D(pool_size=2)(x)
    x = layers.Conv2D(16, kernel_size=5, activation='tanh')(x)
    x = layers.AveragePooling2D(pool_size=2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(120, activation='tanh')(x)
    x = layers.Dense(84, activation='tanh')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model

def build_edgepopup_subnet(global_model, k=0.5):
    inputs = tf.keras.Input(shape=(32, 32, 3))

    w1, b1 = global_model.layers[1].get_weights()
    w2, b2 = global_model.layers[3].get_weights()

    x = SubnetConv2D(w1, b1, 6, 5, k=k, padding='same')(inputs)
    x = layers.Activation('tanh')(x)
    x = layers.AveragePooling2D(pool_size=2)(x)

    x = SubnetConv2D(w2, b2, 16, 5, k=k, padding='valid')(x)
    x = layers.Activation('tanh')(x)
    x = layers.AveragePooling2D(pool_size=2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(120, activation='tanh')(x)
    x = layers.Dense(84, activation='tanh')(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    return tf.keras.Model(inputs, outputs)

# ---------------------------
# Sampling Strategy
# ---------------------------

def least_confidence_sampling(model, unlabeled_pool, n_samples):
    probs = model.predict(unlabeled_pool, verbose=0)
    confidence = np.max(probs, axis=1)
    return np.argsort(confidence)[:n_samples]

# ---------------------------
# Plotting
# ---------------------------

def plot_progress(accs, sizes):
    plt.figure(figsize=(8, 5))
    plt.plot(sizes, accs, 'o-')
    plt.xlabel("Labeled Samples")
    plt.ylabel("Test Accuracy")
    plt.title("Active Learning Progress")
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, "progress.png"))
    plt.close()

def plot_comparison(global_acc, subnet_acc):
    plt.figure(figsize=(8, 5))
    models = ['Global LeNet-5', 'Final Subnet']
    accuracies = [global_acc, subnet_acc]
    plt.bar(models, accuracies)
    plt.ylim([0, 1.0])
    plt.ylabel("Test Accuracy")
    plt.title("Model Comparison after Full Training")
    plt.grid(True, axis='y')
    for i, acc in enumerate(accuracies):
        plt.text(i, acc + 0.01, f"{acc:.4f}", ha='center')
    plt.savefig(os.path.join(output_dir, "model_comparison.png"))
    plt.close()

# ---------------------------
# Active Learning Loop
# ---------------------------

def active_learning(global_model, x_train, y_train, x_test, y_test,
                    k=0.5, init_size=1000, query_size=1000, iterations=10, epochs=10):

    indices = np.arange(len(x_train))
    labeled = np.random.choice(indices, size=init_size, replace=False)
    unlabeled = np.setdiff1d(indices, labeled)

    acc_hist = []
    size_hist = []

    popup_log = []
    final_subnet_model = None  # To store the final subnet model

    for i in range(iterations):
        print(f"\n=== Iteration {i+1} ===")
        x_labeled, y_labeled = x_train[labeled], y_train[labeled]

        model = build_edgepopup_subnet(global_model, k)
        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

        model.fit(x_labeled, y_labeled, validation_split=0.1,
                  epochs=epochs, batch_size=128, verbose=1)

        loss, acc = model.evaluate(x_test, y_test, verbose=0)
        print(f"Test Accuracy: {acc:.4f}")

        # Collect popup score statistics
        score_stats = {}
        for layer in model.layers:
            if isinstance(layer, SubnetConv2D):
                popup_scores = layer.popup_scores.numpy()
                stats = {
                    'min': np.min(popup_scores),
                    'max': np.max(popup_scores),
                    'mean': np.mean(popup_scores),
                    'std': np.std(popup_scores)
                }
                score_stats[layer.name] = stats
                print(f"[{layer.name}] Popup Scores - min: {stats['min']:.4f}, max: {stats['max']:.4f}, mean: {stats['mean']:.4f}, std: {stats['std']:.4f}")

                # Optional: save popup scores for debugging
                np.save(os.path.join(output_dir, f"popup_scores_iter{i+1}_{layer.name}.npy"), popup_scores)

        acc_hist.append(acc)
        size_hist.append(len(labeled))
        popup_log.append(score_stats)

        # Save the model from the final iteration
        if i == iterations - 1:
            final_subnet_model = model
            model.save_weights(final_subnet_weights)
            print(f"Final subnet model saved to {final_subnet_weights}")

        if i < iterations - 1:
            x_pool = x_train[unlabeled]
            indices_new = least_confidence_sampling(model, x_pool, query_size)
            new_samples = unlabeled[indices_new]
            labeled = np.concatenate([labeled, new_samples])
            unlabeled = np.setdiff1d(unlabeled, new_samples)

    # Save accuracy log
    pd.DataFrame({
        "samples": size_hist,
        "accuracy": acc_hist
    }).to_csv(log_file, index=False)

    # Save popup score stats
    pd.DataFrame([
        {"iteration": i+1, "layer": layer, **popup_log[i][layer]}
        for i in range(len(popup_log)) for layer in popup_log[i]
    ]).to_csv(os.path.join(output_dir, "popup_score_stats.csv"), index=False)

    plot_progress(acc_hist, size_hist)
    return acc_hist, size_hist, final_subnet_model


# ---------------------------
# Full Dataset Training
# ---------------------------

def train_models_on_full_dataset(global_model, final_subnet_model, x_train, y_train, x_test, y_test, epochs=20):
    """Train both the global model and final subnet model on the full dataset for comparison"""
    print("\n=== Training Global LeNet-5 on Full Dataset ===")

    # Create a fresh copy of the global model for full training
    global_model_full = build_global_lenet5()
    global_model_full.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    global_model_full.fit(x_train, y_train, epochs=epochs, batch_size=128, validation_split=0.1, verbose=1)
    global_model_full.save_weights(final_global_weights)

    global_loss, global_acc = global_model_full.evaluate(x_test, y_test, verbose=0)
    print(f"Global LeNet-5 Test Accuracy after full training: {global_acc:.4f}")

    print("\n=== Training Final Subnet Model on Full Dataset ===")
    # Reset the final subnet model and train on full dataset
    final_subnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    final_subnet_model.fit(x_train, y_train, epochs=epochs, batch_size=128, validation_split=0.1, verbose=1)

    subnet_loss, subnet_acc = final_subnet_model.evaluate(x_test, y_test, verbose=0)
    print(f"Final Subnet Test Accuracy after full training: {subnet_acc:.4f}")

    # Compare the models
    print("\n=== Model Comparison ===")
    print(f"Global LeNet-5: {global_acc:.4f}")
    print(f"Final Subnet: {subnet_acc:.4f}")
    print(f"Difference: {subnet_acc - global_acc:.4f}")

    # Save comparison results
    comparison_df = pd.DataFrame({
        "model": ["Global LeNet-5", "Final Subnet"],
        "accuracy": [global_acc, subnet_acc]
    })
    comparison_df.to_csv(os.path.join(output_dir, "model_comparison.csv"), index=False)

    # Plot comparison
    plot_comparison(global_acc, subnet_acc)

    return global_acc, subnet_acc


# ---------------------------
# Run the Setup
# ---------------------------

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Subset for speed
num_train_samples = 30000
x_train = x_train[:num_train_samples]
y_train = y_train[:num_train_samples]

# Check if pretrained weights exist
if not os.path.exists(weights_file):
    print("Training and saving global LeNet-5 model...")
    global_lenet5 = build_global_lenet5()
    global_lenet5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    global_lenet5.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.1)
    global_lenet5.save_weights(weights_file)
else:
    print("Loading pretrained LeNet-5 weights...")
    global_lenet5 = build_global_lenet5()
    global_lenet5.load_weights(weights_file)

# Run Active Learning with pretrained global model
acc_history, size_history, final_subnet_model = active_learning(
    global_lenet5, x_train, y_train, x_test, y_test,
    k=0.5, init_size=2000, query_size=2000, iterations=15, epochs=10
)

for i, (n, acc) in enumerate(zip(size_history, acc_history)):
    print(f"Iteration {i+1}: {n} samples - Accuracy: {acc:.4f}")

# Train both models on the full dataset for final comparison
global_acc, subnet_acc = train_models_on_full_dataset(
    global_lenet5, final_subnet_model, x_train, y_train, x_test, y_test, epochs=20
)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
Training and saving global LeNet-5 model...
Epoch 1/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.2846 - loss: 1.9842 - val_accuracy: 0.3563 - val_loss: 1.7957
Epoch 2/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3901 - loss: 1.7349 - val_accuracy: 0.3867 - val_loss: 1.6915
Epoch 3/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4236 - loss: 1.6512 - val_accuracy: 0.4313 - val_loss: 1.6048
Epoch 4/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4483 - loss: 1.5586 - val_accuracy: 0.4423 - val_loss: 1.5394
Epoch 5/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4773 - loss: 1.4786 - val_accuracy: 0.4763 - val_loss: 1.4832
Epoch 6/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5016 - loss: 1.4059 - val_accuracy: 0.4903 - val_loss: 1.4405
Epoch 7/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5258 - loss: 1.3407 - val_accuracy: 0.4853 - val_loss: 1.4235
Epoch

/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['subnet_conv2d/popup_scores', 'subnet_conv2d_1/popup_scores'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 316ms/step - accuracy: 0.1441 - loss: 2.3294 - val_accuracy: 0.2100 - val_loss: 2.1190
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2486 - loss: 2.0487 - val_accuracy: 0.2700 - val_loss: 2.0331
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3121 - loss: 1.9363 - val_accuracy: 0.2650 - val_loss: 1.9786
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3512 - loss: 1.8527 - val_accuracy: 0.3000 - val_loss: 1.9361
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3612 - loss: 1.7947 - val_accuracy: 0.3050 - val_loss: 1.9608
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3855 - loss: 1.7800 - val_accuracy: 0.3200 - val_loss: 1.9879
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3895 - loss: 1.7758 - val_accuracy: 0.2900 - val_loss: 1.9445
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3853 - loss: 1.7613 - val_accuracy: 0.2950 - val_loss: 1.9625
E

/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['subnet_conv2d_2/popup_scores', 'subnet_conv2d_3/popup_scores'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


29/29 ━━━━━━━━━━━━━━━━━━━━ 7s 156ms/step - accuracy: 0.1428 - loss: 2.2771 - val_accuracy: 0.1550 - val_loss: 2.2041
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.2455 - loss: 2.0546 - val_accuracy: 0.2050 - val_loss: 2.1285
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2605 - loss: 2.0187 - val_accuracy: 0.1950 - val_loss: 2.0969
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3011 - loss: 1.9367 - val_accuracy: 0.2350 - val_loss: 2.0680
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3041 - loss: 1.9191 - val_accuracy: 0.2425 - val_loss: 2.0664
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2988 - loss: 1.9112 - val_accuracy: 0.1900 - val_loss: 2.0703
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3209 - loss: 1.9085 - val_accuracy: 0.2850 - val_loss: 2.0245
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3335 - loss: 1.8567 - val_accuracy: 0.2550 - val_loss: 2.0802
E

...

183/183 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.2323 - loss: 2.0590 - val_accuracy: 0.5027 - val_loss: 1.4224
Epoch 2/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3179 - loss: 1.8709 - val_accuracy: 0.5235 - val_loss: 1.3523
Epoch 3/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3464 - loss: 1.8016 - val_accuracy: 0.5538 - val_loss: 1.2532
Epoch 4/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3663 - loss: 1.7396 - val_accuracy: 0.5735 - val_loss: 1.2299
Epoch 5/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3812 - loss: 1.6940 - val_accuracy: 0.5808 - val_loss: 1.1747
Epoch 6/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4073 - loss: 1.6335 - val_accuracy: 0.5900 - val_loss: 1.1530
Epoch 7/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4194 - loss: 1.6018 - val_accuracy: 0.5854 - val_loss: 1.1401
Epoch 8/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4270 - loss: 1.5795 - val_accuracy: 0.5719 - val

/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['subnet_conv2d_26/popup_scores', 'subnet_conv2d_27/popup_scores'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


197/197 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.2467 - loss: 2.0379 - val_accuracy: 0.5989 - val_loss: 1.2656
Epoch 2/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3273 - loss: 1.8527 - val_accuracy: 0.6754 - val_loss: 1.1634
Epoch 3/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3583 - loss: 1.7644 - val_accuracy: 0.7118 - val_loss: 1.0774
Epoch 4/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3869 - loss: 1.7084 - val_accuracy: 0.7143 - val_loss: 1.0210
Epoch 5/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3942 - loss: 1.6584 - val_accuracy: 0.7236 - val_loss: 0.9711
Epoch 6/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4160 - loss: 1.6158 - val_accuracy: 0.7100 - val_loss: 0.9834
Epoch 7/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4344 - loss: 1.5623 - val_accuracy: 0.7368 - val_loss: 0.9064
Epoch 8/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4513 - loss: 1.5183 - val_accuracy: 0.7343 - val

/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['subnet_conv2d_28/popup_scores', 'subnet_conv2d_29/popup_scores'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


211/211 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.2633 - loss: 2.0103 - val_accuracy: 0.7787 - val_loss: 1.0322
Epoch 2/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3476 - loss: 1.8142 - val_accuracy: 0.8210 - val_loss: 0.8891
Epoch 3/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3673 - loss: 1.7452 - val_accuracy: 0.8297 - val_loss: 0.8615
Epoch 4/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3961 - loss: 1.6757 - val_accuracy: 0.8243 - val_loss: 0.7857
Epoch 5/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4161 - loss: 1.6082 - val_accuracy: 0.8507 - val_loss: 0.7123
Epoch 6/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4332 - loss: 1.5728 - val_accuracy: 0.8477 - val_loss: 0.7088
Epoch 7/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4512 - loss: 1.5228 - val_accuracy: 0.8533 - val_loss: 0.6620
Epoch 8/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4580 - loss: 1.4979 - val_accuracy: 0.8570 - val

In [4]:
print("Global LeNet-5 Trainable Parameters:", global_lenet5.count_params())
print("Final Subnet Trainable Parameters:", final_subnet_model.count_params())

Global LeNet-5 Trainable Parameters: 83126
Final Subnet Trainable Parameters: 83104


In [6]:
sample = np.expand_dims(x_test[0], axis=0)  # One sample from the test set

global_pred = global_lenet5.predict(sample, verbose=0)
subnet_pred = final_subnet_model.predict(sample, verbose=0)

print("Global LeNet-5 Prediction:", np.argmax(global_pred))
print("Final Subnet Prediction:", np.argmax(subnet_pred))

print("Prediction Difference (L2 Norm):", np.linalg.norm(global_pred - subnet_pred))


Global LeNet-5 Prediction: 3
Final Subnet Prediction: 8
Prediction Difference (L2 Norm): 0.6391015


In [7]:
def analyze_conv_layer_sparsity(model, model_name="Model"):
    total_params = 0
    zero_params = 0

    print(f"\n=== {model_name} Convolutional Layer Sparsity ===")
    for layer in model.layers:
        if isinstance(layer, SubnetConv2D):  # For subnet model
            weights = layer.base_weights
            mask = layer.get_subnet(tf.abs(layer.popup_scores)).numpy()
            num_total = weights.size
            num_masked = np.sum(mask == 0)

            print(f"[{layer.name}] Total: {num_total}, Masked: {num_masked}, Kept: {num_total - num_masked}")
            total_params += num_total
            zero_params += num_masked

        elif isinstance(layer, tf.keras.layers.Conv2D):  # For global model
            weights = layer.get_weights()
            if weights:
                w = weights[0]
                num_total = w.size
                num_zeros = np.sum(w == 0)

                print(f"[{layer.name}] Total: {num_total}, Zeros: {num_zeros}, Non-Zero: {num_total - num_zeros}")
                total_params += num_total
                zero_params += num_zeros

    print(f"\n=== {model_name} Summary ===")
    print(f"Total Conv Params : {total_params}")
    print(f"Zero/Masked Params: {zero_params}")
    print(f"Non-zero/Kept     : {total_params - zero_params}")
    print(f"Sparsity          : {zero_params / total_params:.2%}")


In [8]:
analyze_conv_layer_sparsity(final_subnet_model, model_name="Final Subnet Model")
analyze_conv_layer_sparsity(global_lenet5, model_name="Global LeNet-5")


=== Final Subnet Model Convolutional Layer Sparsity ===
[subnet_conv2d_28] Total: 450, Masked: 225, Kept: 225
[subnet_conv2d_29] Total: 2400, Masked: 1200, Kept: 1200

=== Final Subnet Model Summary ===
Total Conv Params : 2850
Zero/Masked Params: 1425
Non-zero/Kept     : 1425
Sparsity          : 50.00%

=== Global LeNet-5 Convolutional Layer Sparsity ===
[conv2d] Total: 450, Zeros: 0, Non-Zero: 450
[conv2d_1] Total: 2400, Zeros: 0, Non-Zero: 2400

=== Global LeNet-5 Summary ===
Total Conv Params : 2850
Zero/Masked Params: 0
Non-zero/Kept     : 2850
Sparsity          : 0.00%


In [42]:
for layer in final_subnet_model.layers:
    print(f"{layer.name}: {layer}")


input_layer_15: <InputLayer name=input_layer_15, built=True>
subnet_conv2d_28: <SubnetConv2D name=subnet_conv2d_28, built=True>
activation_28: <Activation name=activation_28, built=True>
average_pooling2d_30: <AveragePooling2D name=average_pooling2d_30, built=True>
subnet_conv2d_29: <SubnetConv2D name=subnet_conv2d_29, built=True>
activation_29: <Activation name=activation_29, built=True>
average_pooling2d_31: <AveragePooling2D name=average_pooling2d_31, built=True>
flatten_15: <Flatten name=flatten_15, built=True>
dense_45: <Dense name=dense_45, built=True>
dense_46: <Dense name=dense_46, built=True>
dense_47: <Dense name=dense_47, built=True>


In [73]:
final_subnet_model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subnet_conv2d_28 (SubnetConv2D) │ (None, 32, 32, 6)      │           450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_28 (Activation)      │ (None, 32, 32, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_30            │ (None, 16, 16, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subnet_conv2d_29 (SubnetConv2D) │ (None, 12, 12, 16)     │         2,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_29 (Activation)      │ (None, 12, 12, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_31            │ (None, 6, 6, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 120)            │        69,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 249,314 (973.89 KB)

 Trainable params: 83,104 (324.62 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 166,210 (649.26 KB)